In [1]:
from pyseqrna import pyseqrna_utils as pu
from pyseqrna import quality_check as qc
from pyseqrna import quality_trimming as qt
from pyseqrna import  aligners as al
from pyseqrna import pyseqrna_stats as ps
from pyseqrna import quantification as quants
from pyseqrna import differential_expression as de
from pyseqrna import pyseqrna_plots as pp
from pyseqrna import multimapped_groups as mmg
import pandas as pd
import dill

During startup - Warning messages:
1: package ‘methods’ was built under R version 4.0.5 
2: package ‘datasets’ was built under R version 4.0.5 
3: package ‘utils’ was built under R version 4.0.5 
4: package ‘grDevices’ was built under R version 4.0.5 
5: package ‘graphics’ was built under R version 4.0.5 
6: package ‘stats’ was built under R version 4.0.5 


In [ ]:
from pyseqrna.pyseqrna_utils import PyseqrnaLogger

log = PyseqrnaLogger(mode='w', log='pp')

log.info("Analysis started")

In [2]:
data = pu.read_input_file("pyseqrna/example/input_Sample_PE.txt", "pyseqrna/example/data/" , paired=True)
samples= data['samples']

[13:48:20]  pyseqrna_utils :: INFO : Reading input samples File 
[13:48:20]  pyseqrna_utils :: INFO : Input file pyseqrna/example/input_Sample_PE.txt read succesfully
[13:48:20]  pyseqrna_utils :: INFO : Combination created succesfully from pyseqrna/example/input_Sample_PE.txt
[13:48:20]  pyseqrna_utils :: INFO : targets dataframe for differenatial created succesfully from pyseqrna/example/input_Sample_PE.txt


In [ ]:
a, b = qc.fastqcRun(sampleDict=samples,paired=True)

In [ ]:
import shutil

shutil.which('fastqc')

In [3]:
targets= data['targets']
comb= data['combinations']

In [ ]:
dill.dump_session("test.pyseqrna")

In [4]:
dill.load_session("test.pyseqrna")

[13:48:25]  __main__ :: INFO : Analysis started at Sun Nov 21 13:48:25 2021


In [8]:
a = mmg.Bam2Bed(outalign)

[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_result

In [12]:
file="pyseqrna/example/data/tair10.gff3"
feature='gene'
from numpy import append
import pysam
import re
import functools
import logging
import pandas as pd
import subprocess

In [13]:
bedFiles = a
gffBED = "pyseqrna/example/data/tair10.gff3.bed"
intbed=[]

Cmd =  'sort -k 1,1 -k2,2n '+ gffBED+' >tmp && mv tmp '+ gffBED

job_id = subprocess.call(Cmd, shell=True)

for sample in bedFiles:

    out= sample.split(".bed")[0]+".intersect.bed"

    intbed.append(out)
    scmd =  'sort -k 1,1 -k2,2n '+ sample+' >tmp && mv tmp '+ sample
    job_id = subprocess.call(scmd, shell=True)
    cmd = "intersectBed -sorted -abam {} -bed -wb -b {} >{}".format(sample,gffBED, out)
    job_id = subprocess.call(cmd, shell=True)

    
listDF=[]

for b in intbed:

    dd = {}
    with open(b, 'r') as fp:
        lines = [line.rstrip() for line in fp]
        for line in lines:
            data = re.split("\s+", line)
            if data[3] not in dd:
                dd[data[3]] = [data[9]]
            else:
                if data[9] != dd[data[3]]:
                    dd[data[3]].append(data[9])

    geneKey = []
    for key in dd:
        geneKey.append('-'.join(sorted(tuple((list(dict.fromkeys(dd[key])))))))  # remove duplicated gene name from list


    df=pd.DataFrame(geneKey)


    df.columns=[b]
    df= pd.DataFrame(df[b].value_counts(dropna=False))
    df.reset_index(level=0, inplace=True)
    df.columns=['gene', b]
    listDF.append(df)

df_final = functools.reduce(lambda left,right: pd.merge(left,right, on='gene', how='outer'), listDF).fillna(0)
# df_final.to_csv("final.txt", sep="\t")
final=df_final.values.tolist()

colna=df_final.columns
newNames = ['MMG', 'Gene']

In [19]:
for key, value in samples.items():
        
        for c in colna:
           
            if c.split("/")[-1].startswith(value[0]):
                newNames.append(key)


    

In [28]:
dk

,MMG,Gene,M1A,M1B,A1A,A1B,V1A,V1B,M6A,M6B,A6A,A6B,V6A,V6B,M12A,M12B,A12A,A12B,V12A,V12B
0,MMG_1,AT2G01010,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
1,MMG_2,AT2G01023,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16


In [27]:
kk=[]
count=0
minCount=10
percentSample=0.5

for f in final:
    
    a=sum(i > minCount for i in f[1:])
    k=len(f[1:])*percentSample
    if a >= k:
        count += 1
        f.insert(0,"MMG_"+str(count))
        kk.append(f)


dk = pd.DataFrame(kk).astype(str).replace('\.0', '', regex=True)
dk.columns=newNames

In [6]:
dk =mmg.countMMG(sampleDict=samples,bamDict=outalign,gff="pyseqrna/example/data/tair10.gff3")

[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_results/M1AAligned.out.bam'
[E::idx_find_and_load] Could not retrieve index file for '/home/nav/Documents/Projects/pyseqrna/pySeqRNA_results/star_result

ValueError: Length mismatch: Expected axis has 0 elements, new values have 20 elements

In [ ]:
pu.make_directory("pySeqRNA_results")

In [ ]:

a = qt.trim_galoreRun(sampleDict=samples, paired=True, slurm=True)

In [ ]:
aligner = al.STAR_Aligner(genome="pyseqrna/example/data/tair10.fasta", slurm=False)

In [ ]:
j = aligner.build_index()

In [ ]:
aligner.check_index()

In [ ]:
outalign, job = aligner.run_Alignment(target=samples, pairedEND=True)

In [ ]:
df= ps.align_stats(sampleDict=samples,trimDict=a[0],bamDict=outalign, pairedEND=True)

In [ ]:
df.to_excel("pySeqRNA_results/alignment_stats.xlsx", index=False)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import seaborn as sns

In [ ]:
sample = list(df['Sample'])
creads = list(df['Input_reads2x'])
iread = list(df['%_Cleaned2x'])
tc= list(100-df['%_Cleaned2x'])

plt.barh(sample,iread,0.6,label="cleaned Reads", color='skyblue')
plt.barh(sample, tc, 0.6,left=iread, label="discarded", color='tomato')
plt.margins(y=0.01)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("% Number of reads", fontsize=18)
plt.ylabel("Samples", fontsize=18)
plt.legend(['Cleaned', 'Discarded'], bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=15)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)

# plt.legend(frameon=False, fontsize=17)



In [ ]:
tc

In [ ]:
a= quants.featureCount(bamDict=outalign, gff="pyseqrna/example/data/arabidopsis10.gff", outDir="pySeqRNA_results", slurm=True)

In [ ]:
counts= pd.read_csv("pySeqRNA_results/Counts_final.txt", sep="\t")

In [ ]:
counts

In [ ]:
targets= data['targets']
comb= data['combinations']

In [ ]:
dd= de.runDESeq2(countDF=counts,targetFile=targets,design='sample', combination=comb)

In [ ]:
import os 
dd.to_excel(os.path.join("pySeqRNA_results","Raw_DEGs_all.xlsx"), index=False)

In [ ]:
dd= pd.read_excel("pySeqRNA_results/Raw_DEGs_all.xlsx")
filtered_DEG = de.degFilter(degDF=dd, CompareList=comb, FDR=0.05, FOLD=1)

In [ ]:
summary= filtered_DEG['summary']

In [ ]:
wd= pd.ExcelWriter(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"))
for key, value in filtered_DEG['filtered'].items():
    value.to_excel(wd,sheet_name=key)
    wd.save()
wd.close()

In [ ]:
pu.getGenes(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"), combinations=comb)

In [ ]:
from pyseqrna import normalize_counts as nc

In [ ]:
rpkm = nc.Normalization(countFile="pySeqRNA_results/Counts_final.txt", featureFile="pyseqrna/example/data/arabidopsis10.gff")

In [ ]:
rpk = rpkm.meanRatioCount()

In [ ]:
rpk.to_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount= pd.read_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount

In [ ]:
from pyseqrna import pyseqrna_plots as pp

In [ ]:
heatmap, ax = pp.plotHeatmap(dd,combinations=comb,num=50, type='deg')

In [ ]:
from pyseqrna import gene_ontology as go

In [ ]:
bdata= go.query('athaliana')

In [ ]:
gdata = go.preprocessBioMart(bdata)

In [ ]:
results = go.enrichGO('athaliana',"pySeqRNA_results/diff_genes/C1-D6.txt")

In [ ]:
results

In [ ]:
df=bdata
import numpy as np
df2 = df[df['GO_ID'].notna()]
gg = list(df2['Gene'])
x = np.array(gg)

bg_count = len(np.unique(x))

bg_count


In [ ]:
cd= pd.read_excel("pySeqRNA_results.1/M1-V6_deg.xlsx")

In [ ]:
gdata

In [ ]:
from pyseqrna import pathway


In [ ]:
dk = pathway.enrichKEGG(file="pySeqRNA_results/diff_genes/C1-D6.txt", species='ath')